In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import math 
import pandas as pd

In [ ]:
#australia state codes use for the url
aus_state = ['NSW','NT','Qld','SA','Tas','Vic','WA']

#headers 
headers_user = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
}

In [ ]:
#listing the categories
def remove3rdslash():
  url = 'https://www.yellowpages.com.au/'
  site = requests.get(url,headers=headers_user)
  soup = BeautifulSoup(site.content, 'html.parser')
  list = soup.find_all('div',class_='rsTmb icon-container')

  #listing index of the 3rd "/"
  inx = []
  for i in list:
    inx.append(i.a['href'].index('/',6))
  
  #listing the href's
  href = []
  for i in list:
     href.append(i.a['href'])
  
  #modified href: removing the strings after the 3rd "/"
  href_mod=[]
  for i,k in zip(href,inx):
    href_mod.append(i[1:k+1])

  #returns a list of categories to be used in url's
  return href_mod
    

In [ ]:
#listed categories
href_mod = remove3rdslash()
href_mod

['find/lawyers-solicitors/',
 'find/dentists/',
 'find/restaurants/',
 'find/plumbers-gas-fitters/',
 'find/electricians-electrical-contractors/',
 'find/mechanics-motor-engineers/',
 'find/hairdressers/',
 'find/builders-building-contractors/',
 'find/accountants-auditors/',
 'find/beauty-salons/',
 'find/vets-veterinary-surgeons/',
 'find/doctors-medical-practitioners/']

In [ ]:
#adding the states on href_mod
href = []
for i in href_mod:
  for k in aus_state:
    href.append(f'{i}{k}')

href[0:3] # 3 example

['find/lawyers-solicitors/NSW',
 'find/lawyers-solicitors/NT',
 'find/lawyers-solicitors/Qld']

In [ ]:
#reconstruct the website address 
def urlReconstruct():
  primary_url = 'https://www.yellowpages.com.au/'
  href_mod2=[]
  for i in href:
    href_mod2.append(f'{primary_url}{i}')
  return href_mod2

In [ ]:
href_mod2 = urlReconstruct()

href_mod2[0:3] # 3 example

['https://www.yellowpages.com.au/find/lawyers-solicitors/NSW',
 'https://www.yellowpages.com.au/find/lawyers-solicitors/NT',
 'https://www.yellowpages.com.au/find/lawyers-solicitors/Qld']

In [ ]:
#check status to be honest not needed you can directly use the '.status_code' directly to reduce time
def status(url):
  site = requests.get(url,headers=headers_user)
  return site.status_code

In [ ]:
#find how many pages in a search
def pageFinder(url):
  site = requests.get(url,headers=headers_user)
  soup = BeautifulSoup(site.content, 'html.parser')
  string_page = soup.find('h2',class_='MuiTypography-root jss265 MuiTypography-body2 MuiTypography-paragraph')
  
  x = string_page.text
  num = x.index(' ')
  pages = math.ceil(int(x[:num])/35)

  if pages > 29:
    pages = 29
  else:
    pages 

  return pages

In [ ]:
#total entries

entries = []
#list the pages of each url
for i in href_mod2:
  entries.append(pageFinder(i))

for i in range(0,len(entries)):
  #for every page there are 35 entries shown in the webpage
  entries[i] = entries[i] * 35

sum(entries)

63140

In [ ]:
#find empty pages
def findemptypage(url):
  empty=[]

  for i in url:
    site = requests.get(i,headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'})
    soup = BeautifulSoup(site.content, 'html.parser')
    string_page = soup.find('h2',class_='MuiTypography-root jss265 MuiTypography-body2 MuiTypography-paragraph')

    if string_page is None:
      empty.append(i)
  
  return empty

In [16]:
empty = findemptypage(href_mod2)

In [17]:
empty

[]

In [18]:
def scrape(url):
  yellowpage = {'office_name':[],'category':[],'address':[], 'general_loc':[],'telephone_num':[], 'website':[]}
  

  #loop thru all states
  for i in url:

      #loop thru pages 


      for j in range(1,pageFinder(i)):

        link = f'{i}/page-{j}'
        site = requests.get(link,headers=headers_user)
        soup = BeautifulSoup(site.content, 'html.parser')

        #check status of website 
        if site.status_code == 200:
          
          scrape = soup.find_all('div',class_='Box__Div-sc-dws99b-0 iOfhmk MuiPaper-root MuiCard-root MuiPaper-elevation1 MuiPaper-rounded')

          for k in range(len(scrape)):
            
            #office Name
            if scrape[k].h3 is not None:
              yellowpage['office_name'].append(scrape[k].h3.text)
            else:
              yellowpage['office_name'].append('0')

            #address
            if scrape[k].find('div',class_='Box__Div-sc-dws99b-0 bvRSwt') is not None:
              yellowpage['address'].append(scrape[k].find('div',class_='Box__Div-sc-dws99b-0 bvRSwt').text)
            else:
              yellowpage['address'].append('0')
            
            #General Location and Category
            if scrape[k].find('p') is not None:
              loc = scrape[k].find('p').text
              x = loc.index(',')
              loc = loc[x+2:]
              yellowpage['general_loc'].append(loc) 
              
              category = scrape[k].find('p').text
              x = category.index(',')
              category = category[:x-1]
              yellowpage['category'].append(category)

            else:
              yellowpage['general_loc'].append('0')
              yellowpage['category'].append('0')

            #telephone number
            if scrape[k].find('button', class_='MuiButtonBase-root MuiButton-root MuiButton-text MuiButton-textPrimary MuiButton-fullWidth') is not None:
              yellowpage['telephone_num'].append(scrape[k].find('button', class_='MuiButtonBase-root MuiButton-root MuiButton-text MuiButton-textPrimary MuiButton-fullWidth').text)
            else:
              yellowpage['telephone_num'].append('0')
            
            #website url
            if scrape[k].find('a',class_='MuiButtonBase-root MuiButton-root MuiButton-text MuiButton-textSecondary MuiButton-fullWidth') is not None:
              yellowpage['website'].append(scrape[k].find('a',class_='MuiButtonBase-root MuiButton-root MuiButton-text MuiButton-textSecondary MuiButton-fullWidth').get('href'))
            else:
              yellowpage['website'].append('0')
          
            
  return yellowpage


# DATA FRAME TO SQL DATABASE


In [19]:
urls = [href_mod2[1],href_mod2[2],href_mod2[67]] #sample url to scrape

In [20]:
dataframe = pd.DataFrame(scrape(urls))

In [21]:
dataframe

,office_name,category,address,general_loc,telephone_num,website
0,Maleys Barristers & Solicitors,Lawyers & Solicitor,"17 Cavenagh St, Darwin, NT, 0800|482m","Darwin, NT 0800",(08) 8981 2266,http://www.maleyslegal.com
1,Maurice Blackburn Lawyers,Lawyers & Solicitor,"Level 1 60 Smith St, Darwin, NT, 0800|260m","Darwin City, NT 0800",1800 703 861,https://www.mauriceblackburn.com.au
2,De Silva Hebron,Lawyers & Solicitor,"47 Knuckey St, Darwin, NT, 0800|346m","Darwin, NT 0800",(08) 8924 4944,http://www.desilva-hebron.com
3,Darwin Family Law,Lawyers & Solicitor,"Level 3/ 9 Cavenagh St, Darwin, NT, 0800|537m","Darwin City, NT 0800",(08) 8946 1666,http://www.darwinfamilylaw.com.au
4,Murphy & Associates Barristers and Solicitors,Lawyers & Solicitor,"25 Flockhart Dr, Marlow Lagoon, NT, 0830|14.04km","Marlow Lagoon, NT 0830",(08) 8932 2062,https://www.facebook.com/Murphy-Associates-Bar...
...,...,...,...,...,...,...
1283,Kameleon Hair & Beauty Salon,Hairdresser,"Shop 2- 118 Hobart Rd, Kings Meadows, TAS, 724...","Kings Meadows, TAS 7249",(03) 6344 9777,0
1284,Boutique Hair & Beauty by Toni,Hairdresser,0,"Westbury, TAS 7303",(03) 6393 2223,0
1285,Compliments Hair & Beauty Salon,Hairdresser,"141 Bligh St, Warrane, TAS, 7018|5.44km","Warrane, TAS 7018",(03) 6244 5763,0
1286,Pandora Hair & Beauty,Hairdresser,"22 John St, Ulverstone, TAS, 7315|213.65km","Ulverstone, TAS 7315",(03) 6425 1573,0


In [22]:
#convert dataframe to csv file
dataframe.to_csv('yellowpage.csv',index=False)

In [ ]:
#intall bit.io python SDK
!pip install bitdotio

In [ ]:
import bitdotio

In [ ]:
b = bitdotio.bitdotio('api_key')#api key was remove due to privacy

In [ ]:
#query string
query_string = """
                CREATE TABLE yellowpage (
                  office_name Varchar,
                  category Varchar,
                  address Varchar,
                  general_loc Varchar,
                  telephone_num Varchar,
                  website Varchar
                )
"""

In [ ]:
#create table 
with b.get_connection('nathanpaulbustamante123/DBNPUB') as conn:
  cursor = conn.cursor()
  cursor.execute(query_string)

In [ ]:
copy_table = f"""
              Copy yellowpage 
              From STDIN 
              With CSV Header Delimiter as ',';
"""

In [ ]:
#copying yellowpage.csv file to database
with open('yellowpage.csv','r') as f:
  with b.get_connection('nathanpaulbustamante123/DBNPUB') as conn:
    cursor = conn.cursor()
    cursor.copy_expert(sql=copy_table,file=f)